In [5]:
def generate_image(payload=None, outfile=None, subfolder=None,test=None):
    if not (payload and outfile and subfolder and test):
        print("Please specify payload, outfile, subfolder,test")
        return False
    else:
        import os
        folder = "data/"+test+"/"+str(subfolder)
        if not os.path.exists(folder):
            os.makedirs(folder)
        response = requests.post(url=f'{url}/sdapi/v1/txt2img', json=payload)

        r = response.json()

        for i in r['images']:
            image = Image.open(io.BytesIO(base64.b64decode(i.split(",",1)[0])))

            png_payload = {
                "image": "data:image/png;base64," + i
            }
            response2 = requests.post(url=f'{url}/sdapi/v1/png-info', json=png_payload)

            pnginfo = PngImagePlugin.PngInfo()
            pnginfo.add_text("parameters", response2.json().get("info"))
            image.save(folder+'/'+str(outfile)+'.png', pnginfo=pnginfo)

In [6]:
import json
import requests
import io
import base64
from PIL import Image, PngImagePlugin

url = "http://127.0.0.1:7860"
character = "tobi_lucky_Michael-InessaGarmash"

payload = {
    "enable_hr": False,
    #"denoising_strength": 0,
    #"firstphase_width": 0,
    #"firstphase_height": 0,
    "prompt": "tobi_lucky, by Michael & Inessa Garmash, photorealistic, natural lighting, masterpiece, high quality, hi res, highres",
    "negative_prompt": "deformed, blurry, bad, disfigured, poor, mutation, mutated, ugly, horror, out of focus, depth of field, focal blur, worst quality, low quality, normal quality",
    #"styles": [
    #  "Good Baseline NSFW",
    #  "Good Baseline NSFW"
    #],
    "seed": 3413886288,
    #"subseed": -1,
    #"subseed_strength": 0,
    #"seed_resize_from_h": -1,
    #"seed_resize_from_w": -1,
    #"sampler_name": "string",
    "batch_size": 1,
    "n_iter": 1,
    "steps": 90,
    "cfg_scale": 7,
    "width": 512,
    "height": 512,
    "restore_faces": False,
    "tiling": False,
    #"eta": 0,
    #"s_churn": 0,
    #"s_tmax": 0,
    #"s_tmin": 0,
    #"s_noise": 1,
    #"override_settings": {},
    "sampler_index": "Euler a"
}

samplers = [
"Euler a",
"Euler",
"LMS",
"Heun",
"DPM2",
"DPM2 a",
"DPM++ 2S a",
"DPM++ 2M",
"DPM fast",
"DPM adaptive",
"LMS Karras",
"DPM2 Karras",
"DPM2 a Karras",
"DPM++ 2S a Karras",
"DPM++ 2M Karras",
"DDIM",
"PLMS"
]

step = 10
while step <= 150:
    for sampler in samplers:
        print(f"Sampler: {sampler} Step: {step}")
        payload["sampler_name"] = sampler
        payload["steps"] = step
        generate_image(payload=payload, outfile=f"{sampler}_{step}",subfolder=character+"-"+str(payload['seed']),test="fullcheck")
    step+=10
print("done")

Sampler: Euler a Step: 10
Sampler: Euler Step: 10
Sampler: LMS Step: 10
Sampler: Heun Step: 10
Sampler: DPM2 Step: 10
Sampler: DPM2 a Step: 10
Sampler: DPM++ 2S a Step: 10
Sampler: DPM++ 2M Step: 10
Sampler: DPM fast Step: 10
Sampler: DPM adaptive Step: 10
Sampler: LMS Karras Step: 10
Sampler: DPM2 Karras Step: 10
Sampler: DPM2 a Karras Step: 10
Sampler: DPM++ 2S a Karras Step: 10
Sampler: DPM++ 2M Karras Step: 10
Sampler: DDIM Step: 10
Sampler: PLMS Step: 10
Sampler: Euler a Step: 20
Sampler: Euler Step: 20
Sampler: LMS Step: 20
Sampler: Heun Step: 20
Sampler: DPM2 Step: 20
Sampler: DPM2 a Step: 20
Sampler: DPM++ 2S a Step: 20
Sampler: DPM++ 2M Step: 20
Sampler: DPM fast Step: 20
Sampler: DPM adaptive Step: 20
Sampler: LMS Karras Step: 20
Sampler: DPM2 Karras Step: 20
Sampler: DPM2 a Karras Step: 20
Sampler: DPM++ 2S a Karras Step: 20
Sampler: DPM++ 2M Karras Step: 20
Sampler: DDIM Step: 20
Sampler: PLMS Step: 20
Sampler: Euler a Step: 30
Sampler: Euler Step: 30
Sampler: LMS Step: 30
